# Projet 7 : Implémentez un modèle de scoring
*Philippe LONJON (janvier 2020)*

---
Ce projet consiste à développer un modèle de scoring, qui donnera une prédiction sur la probabilité de faillite d'un client qui demande un prêt.
Il s'agit d'un problème :
* **Supervisé** : Les étiquettes (Défauts de crédit) sont connus
* **Classification** : Les valeurs à prédire sont des variables qualitatives
---
## Notebook 2 : Fusion des tables, réduction des features
Le notebook comprend :
- la jointure des tables créées dans le notebook 1
- le retrait de variables avec de fortes corrélations
- le retrait de variables avec trop de valeurs manquantes

Les données à utiliser, ont été nettoyées et préparées dans le notebook précédent, et se trouvent dans le répertoire : `data/features`

#### Chargement des packages
Afin de ne pas surcharger les notebooks, les fonctions créées, sont regroupées dans un module ``fonctions08``.

In [1]:
import numpy as np
import pandas as pd

# Librairies machine learning
from sklearn.model_selection import train_test_split

# Module des fonctions du notebook
import fonctions08 as f

# Divers
from time import time, strftime, gmtime
import gc

# Heure démarrage
t0=time()

# Autoreload pour prise en compte des changments dans le module fonctions
%load_ext autoreload
%autoreload 1
%aimport fonctions08

In [2]:
# Constantes
random_state = 1

Pour optimiser la RAM utilisée, nous allons surveiller les variables créées, et supprimer celles devenues inutiles.

In [3]:
# Fonction pour lister les variables créées
def var_active(var_init=[]):
    """
    Input : list of variables to remove from the result
    Return : list of variables created by the script
    """
    var_list = [ var for var in globals() if var not in var_init and not var.startswith('_')]
    return var_list

# Liste des variables intiales à ne pas prendre en compte
var_init = dir()
var_init.append(['var_init', 'var_active'])

## 1. Table principale
La table principale est celle qui regroupe les informations concernant le client et la demande d'emprunt, ainsi que les étiquettes à prédire.

In [4]:
# Chargement dataframe
df_raw = f.import_csv("data/features/features_application.csv", nrows=None)

Memory usage of dataframe is 300.30 MB
Memory usage after optimization is: 62.47 MB
Decreased by 79.2%


Notre jeu de données compte 307 511 entrées.

In [5]:
# Selection lignes / colonnes
df = df_raw.iloc[:, :]
df = df.fillna(np.nan)

# description du dataframe
f.df_info(df, keep=True)
f.check_key(df, ['SK_ID_CURR'])

Shape : 307511 entries and 128 columns
No duplicate values : True
SK_ID_CURR is a primary key : True


In [6]:
# Aperçu des données
df.head()

,SK_ID_CURR,TARGET,application_NAME_CONTRACT_TYPE,application_CODE_GENDER,application_FLAG_OWN_CAR,application_FLAG_OWN_REALTY,application_CNT_CHILDREN,application_AMT_INCOME_TOTAL,application_AMT_CREDIT,application_AMT_ANNUITY,...,application_AMT_REQ_CREDIT_BUREAU_WEEK,application_AMT_REQ_CREDIT_BUREAU_MON,application_AMT_REQ_CREDIT_BUREAU_QRT,application_AMT_REQ_CREDIT_BUREAU_YEAR,application_AMT_DURATION,application_ANNUITY_INCOME_RATIO,application_CREDIT_INCOME_RATIO,application_GOODS_CREDIT_RATIO,application_EMPLOYED_BIRTH_RATIO,application_EXT_SOURCE_ALL
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,1.0,16.46875,0.121948,2.007812,0.863281,0.067322,0.161743
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,36.21875,0.132202,4.789062,0.873047,0.070862,0.466797
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,20.00000,0.099976,2.000000,1.000000,0.011810,0.642578
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,10.53125,0.219849,2.316406,0.949707,0.159912,0.650391
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,23.46875,0.179932,4.222656,1.000000,0.152466,0.322754


Nous identifions les variables qualitatives et quantitatives.

In [7]:
# Listes des variables qualitatives et quantitatives
cols_categorical = df.select_dtypes(include='category').columns.tolist()
cols_numeric = df.select_dtypes(exclude='category').columns.tolist()

print(f"Les données comprennent {len(cols_categorical)} variables qualitatives"
      f" et {len(cols_numeric)} variables quantitatives")

# Distribution de l'étiquette
print("\nDistribution de la variable cible :")
df['TARGET'].value_counts()

Les données comprennent 16 variables qualitatives et 112 variables quantitatives

Distribution de la variable cible :


0    282686
1     24825
Name: TARGET, dtype: int64

### 1.1 Création des données d'entrainement et de test
Nous séparons nos données, le jeu de test ne sera utilisée qu'à la fin pour l'évaluation du modèle.

In [8]:
# Creation d'un jeu d'entrainement et de test
df_train, df_test = train_test_split(df, test_size=0.3,
                                     stratify=df['TARGET'],
                                     random_state=random_state)

### 1.2 Encoding des variables qualitatives
Nous faisons un one-hot encoding des variables qualitatives, sur la base du jeu d'entrainement.

In [9]:
# One hot encoding des variables qualitatives des jeux de données
df_train_encoded = pd.get_dummies(df_train)
df_test_encoded = pd.get_dummies(df_test)

# Alignement des colonnes de du jeu de test sur celui du train
df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join = 'left', axis=1)

print(df_train_encoded.shape)
print(df_test_encoded.shape)

(215257, 249)
(92254, 249)


### 1.3 Gestion de la RAM
Nous listons les variables, puis supprimons celles qui ne seront plus utilisées.

In [10]:
# Variables à supprimer pour libérer de la mémoire
var_active(var_init=var_init)

['var_init',
 'df_raw',
 'df',
 'cols_categorical',
 'cols_numeric',
 'df_train',
 'df_test',
 'df_train_encoded',
 'df_test_encoded']

In [11]:
# Mémoire avant suppression
print(f"Mémoire utilisée : {f.memory_usage():.0f} Mb")

# Suppression des variables pour libérer de la mémoire
gc.enable()
del df_raw, df, cols_categorical, cols_numeric
del df_train, df_test
gc.collect()

# Mémoire après suppression
print(f"Mémoire utilisée : {f.memory_usage():.0f} Mb")

svmem(total=34322874368, available=30244098048, percent=11.9, used=4078776320, free=30244098048)
Mémoire utilisée : 438 Mb
svmem(total=34322874368, available=30376243200, percent=11.5, used=3946631168, free=30376243200)
Mémoire utilisée : 309 Mb


## 2. Jointures des tables complémentaires
Les jeux de données d'entrainement et de test étant crées, on peut leur joindre les informations des tables complémentaires.<br>
On commence par charger les tables :

In [12]:
# Chargement fichiers
bureau = f.import_csv("data/features/features_bureau.csv")
bureau_balance = f.import_csv("data/features/features_bureau_balance.csv")
previous_application = f.import_csv("data/features/features_previous_application.csv")
cash_balance = f.import_csv("data/features/features_cash_balance.csv")
card_balance = f.import_csv("data/features/features_card_balance.csv")
installments_payments = f.import_csv("data/features/features_installments_payments.csv")

Memory usage of dataframe is 237.98 MB
Memory usage after optimization is: 65.91 MB
Decreased by 72.3%
Memory usage of dataframe is 46.66 MB
Memory usage after optimization is: 11.37 MB
Decreased by 75.6%
Memory usage of dataframe is 959.14 MB
Memory usage after optimization is: 218.78 MB
Decreased by 77.2%
Memory usage of dataframe is 244.44 MB
Memory usage after optimization is: 72.04 MB
Decreased by 70.5%
Memory usage of dataframe is 281.27 MB
Memory usage after optimization is: 109.33 MB
Decreased by 61.1%
Memory usage of dataframe is 443.03 MB
Memory usage after optimization is: 157.72 MB
Decreased by 64.4%


In [13]:
# Mémoire démarrage
print(f"Mémoire utilisée : {f.memory_usage():.0f} Mb")

svmem(total=34322874368, available=29753827328, percent=13.3, used=4569047040, free=29753827328)
Mémoire utilisée : 957 Mb


### 2.1 Jointure des tables aux données d'entrainement
On peut alors faire la jointure sur les données d'entrainement.

In [14]:
# Jointure donénes entrainement
df_train = df_train_encoded\
    .merge(bureau, on='SK_ID_CURR', how='left')\
    .merge(bureau_balance, on='SK_ID_CURR', how='left')\
    .merge(previous_application, on='SK_ID_CURR', how='left')\
    .merge(cash_balance, on='SK_ID_CURR', how='left')\
    .merge(card_balance, on='SK_ID_CURR', how='left')\
    .merge(installments_payments, on='SK_ID_CURR', how='left')

### 2.2 Jointure des tables aux données de test
On fait également la jointure sur les données de test.

In [15]:
#Jointure données test
df_test = df_test_encoded\
    .merge(bureau, on='SK_ID_CURR', how='left')\
    .merge(bureau_balance, on='SK_ID_CURR', how='left')\
    .merge(previous_application, on='SK_ID_CURR', how='left')\
    .merge(cash_balance, on='SK_ID_CURR', how='left')\
    .merge(card_balance, on='SK_ID_CURR', how='left')\
    .merge(installments_payments, on='SK_ID_CURR', how='left')

### 2.3 Finalisation des données d'entrée du modèle
On s'assure que les données de test sont bien alignées sur les données d'entrainement.

In [16]:
print(df_train.shape)
print(df_test.shape)

# Alignement des colonnes de du jeu de test sur celui du train
df_train, df_test = df_train.align(df_test, join = 'left', axis=1)

(215257, 1358)
(92254, 1358)


On extrait les identifiants clients et les étiquettes qui ne seront pas des variables d'entrée pour le modèle à venir.

In [17]:
# Indentifiant en index
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')

# Extraction des étiquettes
train_target = df_train[['TARGET']]
test_target = df_test[['TARGET']]

# Suppression des étiquettes des features
df_train = df_train.drop(columns=['TARGET'], axis=1)
df_test = df_test.drop(columns=['TARGET'], axis=1)

### 2.4 Gestion RAM
On supprime les variables qui ne seront plus utilisées

In [18]:
# Mémoire avant suppression
print(f"Mémoire utilisée : {f.memory_usage():.0f} Mb")

# Suppression des variables pour libérer de la mémoire
gc.enable()
del df_train_encoded, df_test_encoded
del bureau, bureau_balance, previous_application
del cash_balance, card_balance, installments_payments
gc.collect()

# Mémoire après suppression
print(f"Mémoire utilisée : {f.memory_usage():.0f} Mb")

svmem(total=34322874368, available=27510366208, percent=19.8, used=6812508160, free=27510366208)
Mémoire utilisée : 3082 Mb
svmem(total=34322874368, available=28923994112, percent=15.7, used=5398880256, free=28923994112)
Mémoire utilisée : 1734 Mb


# 3. Retrait des variables avec forte corrélation
On retire les variables ayant des corrélations supérieures à 0.9, qui apportent peu d'information pour le
modèle, mais augmentent sa complexité.

In [19]:
# Niveau de corrélation à partir duquel on supprime les variables corrélées
threshold = 0.9

# Matrice de corrélation
corr_matrix = df_train.corr().abs()

# Triangle supérieur de la matrice de corrélation
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Colonnes corrélées à supprimer
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print(f"Il y a {len(to_drop)} colonnes corrélées à supprimer.")

Il y a 541 colonnes corrélées à supprimer.


In [20]:
df_train = df_train.drop(columns = to_drop, axis=1)
df_test = df_test.drop(columns = to_drop, axis=1)

In [21]:
print(df_train.shape)
print(df_test.shape)

(215257, 815)
(92254, 815)


# 4. Suppression des features avec trop de valeurs manquantes
On va également supprimer les variables ayant trop de valeurs manquantes.

In [22]:
# Valeurs manquantes 
train_missing = (df_train.isnull().sum() / len(df_train)).sort_values(ascending = False)
train_missing.head()

previous_RATE_INTEREST_PRIVILEGED_min               0.984800
card_balance_AMT_PAYMENT_CURRENT_min_min            0.801423
card_balance_AMT_DRAWINGS_OTHER_CURRENT_mean_min    0.801103
card_balance_AMT_DRAWINGS_ATM_CURRENT_max_min       0.801103
card_balance_CNT_DRAWINGS_OTHER_CURRENT_max_min     0.801103
dtype: float64

On décide de garder les variables avec moins de 75% de valeurs manquantes.

In [23]:
# Identify missing values above threshold
train_missing = train_missing[train_missing > 0.75].index

print(f"Il y a {len(train_missing)} colonnes avec des valeur manquantes à supprimer.")

Il y a 16 colonnes avec des valeur manquantes à supprimer.


In [24]:
df_train = df_train.drop(columns = train_missing, axis=1)
df_test = df_test.drop(columns = train_missing, axis=1)

print(df_train.shape)
print(df_test.shape)

(215257, 799)
(92254, 799)


# 5. Sauvegarde des données pour les modèles
On sauvegarde les données de de test et d'entrainement.

In [25]:
# Alignement des colonnes de du jeu de test sur celui du train avant sauvegarde
df_train, df_test = df_train.align(df_test, join = 'left', axis=1)

# sauvegarde des dataframes
df_train.to_csv("data/features/train_features.csv")
df_test.to_csv("data/features/test_features.csv")

train_target.to_csv("data/features/train_target.csv")
test_target.to_csv("data/features/test_target.csv")

In [26]:
t1 = time()
print("computing time : {:8.6f} sec".format(t1-t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1-t0)))

computing time : 1488.906388 sec
computing time : 00:24:48
